In [22]:
# Cell 1: Import necessary libraries
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [72]:
df = pd.read_excel('/content/combined_dataset_with_embeddings.xlsx')

In [73]:
df.head()

,ID,Name,Role,Transcript,Resume,decision,Reason for decision,Job Description,Transcript_length,Resume_length,Job_Description_length,Transcript_Resume_Score,Transcript_JobDescription_Score,Resume_JobDescription_Score,Final_Match_Score,Transcript_embeddings,Resume_embeddings,Job_Description_embeddings
0,uppaup,alice smith,software engineer,heres a simulated interview for a software eng...,heres a sample resume for alice smith\n\nalice...,rejected,unsatisfactory references or background check,here is a comprehensive job description for a ...,3101,2599,3857,0.128843,0.159541,0.267942,0.185442,[-1.27699479e-01 2.35646144e-01 -2.42982343e-...,[-6.07043445e-01 2.15680733e-01 -2.49858886e-...,[-5.35873592e-01 -4.82691169e-01 2.11648270e-...
1,uppaup,hank brown,software engineer,heres a simulated interview for a software eng...,heres a resume for hank brown a selected candi...,selected,growth mindset and adaptability,here is a job description for a software engin...,4965,2634,1066,0.268331,0.190271,0.268425,0.242342,[-4.01487261e-01 9.19529703e-03 -2.97915787e-...,[-9.60681498e-01 -1.35371417e-01 -2.07896963e-...,[-1.42806888e-01 -1.84554830e-01 -1.43176287e-...
2,uppaup,bob jones,data scientist,heres a simulated interview for a data scienti...,heres a sample resume for bob jones who applie...,rejected,inadequate communication or interpersonal skills,here is a comprehensive job description for a ...,2803,3050,3105,0.311734,0.285505,0.466292,0.354511,[-3.48792613e-01 2.01288387e-02 -2.33644307e-...,[-9.85215783e-01 -6.83838353e-02 6.52836869e-...,[-6.39032066e-01 -1.17160745e-01 2.43715808e-...
3,uppaup,bob miller,software engineer,heres a simulated interview for a software eng...,heres a sample resume for bob miller a softwar...,selected,strong cultural fit,here is a sample job description for a softwar...,4119,2491,3354,0.226898,0.259491,0.326965,0.271118,[-5.01133323e-01 1.24364235e-01 -2.73392916e-...,[-7.35497475e-01 3.17984790e-01 -3.56828459e-...,[-7.92037129e-01 1.55533820e-01 -2.63567537e-...
4,uppaup,ivy jones,data engineer,heres a simulated interview for a data enginee...,heres a sample resume for ivy jones\n\nivy jon...,rejected,lack of relevant skills or experience,here is a sample job description for a data en...,3321,2864,3843,0.361991,0.420489,0.588236,0.456905,[-2.76312590e-01 1.00067988e-01 -1.50444090e-...,[-6.41999722e-01 9.42382291e-02 4.63643037e-...,[-6.07518435e-01 1.02332525e-01 2.60015577e-...


In [74]:
# 1. Calculate TF-IDF and Similarity Scores
vectorizer = TfidfVectorizer()
vectorizer.fit(df['Job Description'] + df['Resume'])

job_description_vectors = vectorizer.transform(df['Job Description'])
resume_vectors = vectorizer.transform(df['Resume'])

similarity_scores = cosine_similarity(job_description_vectors, resume_vectors)
df['similarity_scores'] = similarity_scores.diagonal()

In [77]:
# 2. Analyze Similarity Across Roles
role_similarity = df.groupby('Role')['similarity_scores'].mean()
print("Average Similarity Scores by Role:\n")
role_similarity

Average Similarity Scores by Role:



,similarity_scores
Role,
ai engineer,0.609176
business analyst,0.516215
cloud architect,0.542400
content writer,0.765457
cybersecurity specialist,0.611475
data analyst,0.128020
data engineer,0.257870
data scientist,0.238902
database administrator,0.576657


In [78]:
# 3. Set Similarity Thresholds (Example)
threshold = 0.5  # Adjust this threshold based on your analysis
df['Decision'] = df['similarity_scores'].apply(lambda score: 'Selected' if score >= threshold else 'Rejected')

In [79]:

# Define bins and labels
bins = [0, 0.1, 0.2, 0.4, 0.6, 0.8, 1]
labels = ['0-0.1', '0.1-0.2', '0.2-0.4', '0.4-0.6', '0.6-0.8', '0.8-1']

# Create a new column with binned values
df['similarity_bin'] = pd.cut(df['Resume_JobDescription_Score'], bins=bins, labels=labels, include_lowest=True)

# Group data and analyze distribution, including 'Rejected'
distribution = df.groupby(['similarity_bin', 'Decision', 'Role']).size().unstack(fill_value=0)

<ipython-input-79-4dc6c41d6ea6>:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  distribution = df.groupby(['similarity_bin', 'Decision', 'Role']).size().unstack(fill_value=0)


In [82]:
# Print the distribution
distribution

Role                     ai engineer  business analyst  cloud architect  \
similarity_bin Decision                                                   
0-0.1          Rejected            0                 0                0   
               Selected            0                 0                0   
0.1-0.2        Rejected            0                 0                0   
               Selected            0                 0                0   
0.2-0.4        Rejected            2                 0                0   
               Selected            1                 0                0   
0.4-0.6        Rejected            0                 4                1   
               Selected           10                 3                7   
0.6-0.8        Rejected            0                 0                0   
               Selected            0                 4                0   
0.8-1          Rejected            0                 0                0   
               Selected            0                 0                0   

Role                     content writer  cybersecurity specialist  \
similarity_bin Decision                                             
0-0.1          Rejected               0                         0   
               Selected               0                         0   
0.1-0.2        Rejected               0                         0   
               Selected               0                         0   
0.2-0.4        Rejected               0                         1   
               Selected               0                         0   
0.4-0.6        Rejected               0                         0   
               Selected               5                         8   
0.6-0.8        Rejected               0                         0   
               Selected               0                         0   
0.8-1          Rejected               0                         0   
               Selected               0                         0   

Role                     data analyst  data engineer  data scientist  \
similarity_bin Decision                                                
0-0.1          Rejected           139            143             109   
               Selected             0              0               0   
0.1-0.2        Rejected            85             71             105   
               Selected             0              0               0   
0.2-0.4        Rejected            27            120             246   
               Selected             0              0               0   
0.4-0.6        Rejected             3             87              91   
               Selected             3              8              25   
0.6-0.8        Rejected             0              9               0   
               Selected             4             61               3   
0.8-1          Rejected             0              0               0   
               Selected             0              0               0   

Role                     database administrator  devops engineer  ...  \
similarity_bin Decision                                           ...   
0-0.1          Rejected                       0                0  ...   
               Selected                       0                0  ...   
0.1-0.2        Rejected                       0                0  ...   
               Selected                       0                0  ...   
0.2-0.4        Rejected                       0               10  ...   
               Selected                       0                3  ...   
0.4-0.6        Rejected                       1                0  ...   
               Selected                       7                3  ...   
0.6-0.8        Rejected                       0                0  ...   
               Selected                       1                0  ...   
0.8-1          Rejected                       0                0  ...   
               Selected 